## LCMS Data Processing
* Data extraction from mdb's to excel workbook.
* Extraction of GPS table from mdb's to excel workbook.
* Extraction of Texture table from mdb's into excel workbook.
* Extraction of Rut table from mdb's into excel workbook.
* Extraction of Rough table from mdb's into excel workbook.

In [3]:
# importing packages
import pyodbc
import os
import pandas as pd

mdb_directory = r"D:\_1.Project\AACM\Pavement\Data\MDB_Processed" # Define the directory containing MDB files

# Define the paths to the Excel files
o_gps = r"D:\_1.Project\AACM\Pavement\Excel Workbook\Mdb_data_Workbook\E_GPS.xlsx"
o_rut = r"D:\_1.Project\AACM\Pavement\Excel Workbook\Mdb_data_WorkbookE_Rut.xlsx"
o_rough = r"D:\_1.Project\AACM\Pavement\Excel Workbook\Mdb_data_Workbook\E_Rough.xlsx"
o_text = r"D:\_1.Project\AACM\Pavement\Excel Workbook\Mdb_data_Workbook\E_Text.xlsx"


def read_tables_by_prefix(mdb_file, table_prefix, survey_id): # Define the function to read tables in MDB
    conn_str = f"Driver={{Microsoft Access Driver (*.mdb, *.accdb)}};Dbq={mdb_file};"
    connection = pyodbc.connect(conn_str)
    cursor = connection.cursor()

    tables = []
    for table_info in cursor.tables(tableType='TABLE'):
        table_name = table_info.table_name
        if table_name.startswith(table_prefix):
            tables.append(table_name)

    data = pd.DataFrame()
    for table in tables:
        query = f"SELECT * FROM [{table}]"
        table_data = pd.read_sql(query, connection)
        table_data['survey_id_1'] = survey_id  # Add a new column for Survey_ID and populate with survey_id
        data = pd.concat([data, table_data], ignore_index=True)

    connection.close()
    return data


def read_mdb(x): # Define the function to read MDB files
    mdb_files = []
    for root, dirs, files in os.walk(x):
        for file in files:
            if file.endswith(".mdb"):
                mdb_files.append(os.path.join(root, file))
    return mdb_files


mdb_files = read_mdb(mdb_directory)  # Call the function to read MDB files

# Create empty dataframes to store data
GPS = pd.DataFrame()
Rut = pd.DataFrame()
Rough = pd.DataFrame()
Texture = pd.DataFrame()


for mdb_file in mdb_files:  # Loop through each MDB file and extract data
    # Extract data for each table type
    gps = read_tables_by_prefix(mdb_file, "GPS_Processed", os.path.splitext(os.path.basename(mdb_file))[0])
    rut = read_tables_by_prefix(mdb_file, "LCMS_Rut_Processed", os.path.splitext(os.path.basename(mdb_file))[0])
    rough = read_tables_by_prefix(mdb_file, "LCMS_Rough_Processed", os.path.splitext(os.path.basename(mdb_file))[0])
    text = read_tables_by_prefix(mdb_file, "LCMS_Texture_Processed", os.path.splitext(os.path.basename(mdb_file))[0])
    
    # Merge the extracted data with the respective dataframe
    GPS = pd.concat([GPS, gps], ignore_index=True)
    Rut = pd.concat([Rut, rut], ignore_index=True)
    Rough = pd.concat([Rough, rough], ignore_index=True)
    Texture = pd.concat([Texture, text], ignore_index=True)

# Save each dataframe to Excel
GPS.to_excel(o_gps, index=False)
Rut.to_excel(o_rut, index=False)
Rough.to_excel(o_rough, index=False)
Texture.to_excel(o_text, index=False)

print("Data saved to Excel files successfully!!")


Data saved to Excel files successfully!!


### Converting excel's into tables
* creating table of each excels
* join coordinates from gps to each table
* creating points from each table
* creating gps line

In [4]:
#import arcpy package
import arcpy,os
PCI_schema = {}  #Dictionary to store PCI schema 
IRI_schema = {}  #Dictionary to store IRI schema
pci_path = r"D:\_1.Project\AACM\Schema_Boundaries\PAVEMENT SCHEMA\GDB\AACM_PAVEMENT_GIS_DATABASE.gdb\PCI"
iri_path = r"D:\_1.Project\AACM\Schema_Boundaries\PAVEMENT SCHEMA\GDB\AACM_PAVEMENT_GIS_DATABASE.gdb\IRI"

fields_pci = arcpy.ListFields(pci_path) #iteration over pci feature template to get fields
for field in fields_pci:
    if not any(substring in field.name for substring in ["SHAPE", "SHAPE_Length", "OBJECTID"]):
        IRI_schema[field.name] = (field.aliasName, field.type, field.length) # appending to dictionary


fields_iri = arcpy.ListFields(iri_path)  #iteration over iri feature template to get fields
for field in fields_iri:
    if not any(substring in field.name for substring in ["SHAPE", "SHAPE_Length", "OBJECTID"]):
        PCI_schema[field.name] = (field.aliasName, field.type, field.length) # appending to dictionary
                 

WB_path = r"D:\_1.Project\AACM\Pavement\Excel Workbook\Mdb_data_Workbook" # excel workbooks path
arcpy.env.workspace = r"D:\_1.Project\AACM\Pavement\GDB\Data_Extraction.gdb" #output workspace


for root, dirs, files in os.walk(WB_path): #converting workbooks into respecitve tables
    for file in files:
        excel_file_path = os.path.join(root, file)
        table_name = "T" + file[1:-5]
        table_output_path = os.path.join(arcpy.env.workspace, table_name)
        arcpy.conversion.ExcelToTable(excel_file_path, table_output_path)
        table_name = arcpy.ValidateTableName(table_name)
        arcpy.management.AddField(table_name, "link_1", "TEXT")
        expr = "!survey_id_1! + '_' + str(!CHAINAGE!)"
        arcpy.management.CalculateField(table_name, "link_1", expr, "PYTHON3")
            
            
for table in arcpy.ListTables(): #adding coordinates to tables
    if table != "T_GPS":
        arcpy.management.JoinField(table, 'link_1', 'T_GPS', 'link_1', ['LONGITUDE','LATITUDE'])         


for table in arcpy.ListTables(): # table to gps point and gps line
    crs = arcpy.SpatialReference(4326)
    out_features = "P"+ table[1:]
    arcpy.management.XYTableToPoint(table,out_features, 'LONGITUDE','LATITUDE',"",crs)
    arcpy.management.PointsToLine("P_GPS","L_GPS","survey_id_1")
                
for f_name, f_props in PCI_schema.items(): #importing gps line into AACM schema 
    name = f_name
    alias = f_props[0]
    data_type = f_props[1]
    length = f_props[2]
    arcpy.management.AddField("L_GPS", field_name=name, field_alias=alias, field_type=data_type, field_length=length)


crs_utm = arcpy.SpatialReference(32640) #reprojecting into utm
arcpy.management.Project("L_GPS", "L_GPS_utm", crs_utm)
arcpy.AlterAliasName("L_GPS_utm","L_GPS_utm")

print("successfully converted and exported to GPS,IRI,Rough and Texture feature classes!")

successfully converted and exported to GPS,IRI,Rough and Texture feature classes!


## Attribute Data population
* From direction is populated
* To direction is populated

In [6]:
import math
import arcpy

arcpy.env.workspace = r"D:\_1.Project\AACM\Pavement\GDB\Data_Extraction.gdb"
fc = "pci_test"

def get_compass_direction(azimuth): # Function to calculate compass direction based on azimuth angle
    directions = ["N", "NE", "E", "SE", "S", "SW", "W", "NW"]
    sector_size = 360 / len(directions)
    sector_index = round(azimuth / sector_size) % len(directions)
    return directions[sector_index]

with arcpy.da.UpdateCursor(fc, ["SHAPE@", "TODIR", "FROMDIR"]) as cursor: # Update directions in the feature class
    for row in cursor:
        line = row[0]
        start_point = line.firstPoint
        end_point = line.lastPoint
        azimuth_start = math.atan2(end_point.X - start_point.X, end_point.Y - start_point.Y)
        azimuth_end = math.atan2(start_point.X - end_point.X, start_point.Y - end_point.Y)
        azimuth_start_degrees = math.degrees(azimuth_start)
        azimuth_end_degrees = math.degrees(azimuth_end)
        row[1] = get_compass_direction(azimuth_start_degrees)  # Populate From_Dir
        row[2] = get_compass_direction(azimuth_end_degrees)  # Populate To_Dir
        cursor.updateRow(row)

with arcpy.da.UpdateCursor(fc, ["TODIR", "ORIENT"]) as cursor: # Update directions in the feature class
    for row in cursor: 
        if row[0] in ("N", "E", "NE","NW"):
            row[1] = 5
        elif row[0] in ("S", "W", "SE" ,"SW"):
            row[1] = 6
        else:
            row[1] = 0
        cursor.updateRow(row)

print("Successfully populated From_Dir,o_Dir fields and Orientation")


Successfully populated From_Dir,o_Dir fields and Orientation


* district no
* district english name
* district arabic name

In [10]:
arcpy.env.workspace = r"D:\_1.Project\AACM\Pavement\GDB\Data_Extraction.gdb"
fc = "pci_test"
district_dict = {}
district_bound = r"D:\_1.Project\AACM\Schema_Boundaries\SCOPE_WORK\SCOPE_WORK.gdb\DISTRICT_BOUND"
arcpy.analysis.SpatialJoin(fc,district_bound,"fc_join","JOIN_ONE_TO_ONE","KEEP_ALL")

with arcpy.da.SearchCursor("fc_join", ["TARGET_FID", "DISTRICT_NO", "DISTRICT_NAME_ENG"]) as cursor:
    for row in cursor:
        object_id = row[0]
        district_no = row[1]
        district_name = row[2]
        district_dict[object_id] = (district_no, district_name)

# Update the "DISTRICT" and "COMMENTS" fields based on values in the dictionary
with arcpy.da.UpdateCursor(fc, ["OBJECTID", "RIGHTOFWAYWIDTH","DISTRICT"]) as cursor:
    for row in cursor:
        object_id = row[0]
        if object_id in district_dict:
            row[1], row[2] = district_dict[object_id]
            cursor.updateRow(row)

print("suceesfully populated district name and district no's into pci lines!!!")

suceesfully populated district name and district no's into pci lines!!!
